# Connecting Google Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Connecting Kaggle

In [2]:
# Install kaggle
!pip install -q kaggle

In [3]:
# upload kaggle.json file which we downloaded
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"arbazkhancs","key":"140450ae4ad01edb47d18c912d681ad3"}'}

In [4]:
# Create A Kaggle Folder
! mkdir ~/.kaggle

In [5]:
# Copy the kaggle.json file to the above folder
! cp kaggle.json ~/.kaggle/

In [6]:
# Permission for the json to act
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
# to list all kaggle datasets
! kaggle datasets list

ref                                                     title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                      Vehicle Sales Data                             19MB  2024-02-21 20:16:17           4690         77  1.0              
nelgiriyewithana/apple-quality                          Apple Quality                                 170KB  2024-01-11 14:31:07          24137        519  1.0              
kanchana1990/spotifys-long-hits-2014-2024               Spotify's Long Hits (2014-2024) 🎶              38KB  2024-02-23 12:39:02           1742         44  1.0              
xontoloyo/data-penjualan-zara                           ZARA Sales                                     17KB  2024-02-27 05:37:53  

# Downloading the Dataset

In [8]:
! kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 138MB/s]


In [9]:
# unzip the downloaded file
! unzip twitter-entity-sentiment-analysis.zip

Archive:  twitter-entity-sentiment-analysis.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


# Text Preprocessing

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [11]:
df = pd.read_csv("twitter_training.csv", header=None, names=['ID', 'Game', 'Sentiment', 'Tweet'])

In [12]:
df.head()

,ID,Game,Sentiment,Tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         74682 non-null  int64 
 1   Game       74682 non-null  object
 2   Sentiment  74682 non-null  object
 3   Tweet      73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [14]:
df.shape

(74682, 4)

In [15]:
# Select only required columns
df = df.drop(columns=["ID", "Game"])

In [16]:
df.head()

,Sentiment,Tweet
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...


# Preprocessing

In [46]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re


# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

class preProcess:
    def __init__(self, Dataframe):
        self.Dataframe = Dataframe.copy()
        # Drop all null values
        self.Dataframe.dropna(inplace=True)

        self.tweet = []
        self.sentiments = None


    def process_Sentiment(self):
        mapping = {}
        count = 0
        for i in self.Dataframe["Sentiment"].unique():
            mapping[i] = count
            count += 1

        print(mapping)
        self.Dataframe["Sentiment"] = self.Dataframe["Sentiment"].apply(lambda x: mapping[x])
        self.sentiments = np.array(self.Dataframe["Sentiment"])
        return self.sentiments


    def process_Tweet(self):
        processed_tweets = []

        for line in tqdm(self.Dataframe["Tweet"]):
            # Remove URLs
            line = re.sub(r'http\S+|www\S+', '', line)

            # Remove punctuations except apostrophes
            line = re.sub(r'[^\w\s\']', '', line)

            # Remove tokens < 2
            filtered_tokens = [token for token in line.split() if len(token) > 1]

            processed_line = ' '.join(filtered_tokens)

            processed_tweets.append(processed_line)

        self.Dataframe["Tweet"] = processed_tweets


    def creat_Tokenizer(self):
        # creating tokenizer
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(self.Dataframe["Tweet"])
        print(tokenizer.word_index)

        self.tweet = tokenizer.texts_to_sequences(self.Dataframe["Tweet"])
        print(self.tweet[0])


    def do_pad_sequence(self):
        # pad sequences
        self.tweet = pad_sequences(self.tweet, padding='post')  # Pad sequences

        print(self.tweet[0])  # Print the first padded sequence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
# create object of preProcess class
obj = preProcess(df)

In [48]:
# process the Sentiment
obj.process_Sentiment()

{'Positive': 0, 'Neutral': 1, 'Negative': 2, 'Irrelevant': 3}


array([0, 0, 0, ..., 0, 0, 0])

In [49]:
# process the tweets
obj.process_Tweet()

100%|██████████| 73996/73996 [00:00<00:00, 92036.09it/s]

0        im getting on borderlands and will murder you all
1           am coming to the borders and will kill you all
2          im getting on borderlands and will kill you all
3         im coming on borderlands and will murder you all
4        im getting on borderlands and will murder you ...
                               ...                        
74677    Just realized that the Windows partition of my...
74678    Just realized that my Mac window partition is ...
74679    Just realized the windows partition of my Mac ...
74680    Just realized between the windows partition of...
74681    Just like the windows partition of my Mac is l...
Name: Tweet, Length: 73996, dtype: object


In [50]:
# create tokenizer
obj.creat_Tokenizer()

{'the': 1, 'to': 2, 'and': 3, 'of': 4, 'is': 5, 'for': 6, 'in': 7, 'this': 8, 'it': 9, 'you': 10, 'on': 11, 'my': 12, 'that': 13, 'with': 14, 'game': 15, 'so': 16, 'me': 17, 'have': 18, 'just': 19, 'but': 20, 'not': 21, 'are': 22, 'be': 23, 'all': 24, 'was': 25, 'at': 26, 'like': 27, 'out': 28, 'from': 29, 'your': 30, 'now': 31, 'get': 32, 'we': 33, 'as': 34, 'they': 35, 'has': 36, 'if': 37, 'one': 38, 'do': 39, 'good': 40, 'about': 41, 'can': 42, 'play': 43, 'no': 44, 'will': 45, 'an': 46, 'really': 47, 'when': 48, 'love': 49, 'new': 50, 'up': 51, "i'm": 52, 'what': 53, 'more': 54, 'time': 55, 'by': 56, 'how': 57, 'some': 58, 'people': 59, 'or': 60, 'johnson': 61, 'why': 62, 'see': 63, 'shit': 64, "it's": 65, 'its': 66, 'been': 67, 'best': 68, 'still': 69, 'got': 70, 'games': 71, 'great': 72, 'even': 73, 'playing': 74, 'who': 75, 'because': 76, 'xbox': 77, 'fucking': 78, 'go': 79, 'fuck': 80, 'facebook': 81, 'google': 82, 'them': 83, 'dead': 84, 'rhandlerr': 85, 'had': 86, 'amazon': 8

In [51]:
# make all same length
obj.do_pad_sequence()

[ 100  152   11  121    3   45 1674   10   24    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]


# Training Dataset

In [52]:
X_train = obj.tweet
y_train = obj.sentiments

In [53]:
X_train.shape, y_train.shape

((73996, 150), (73996,))

In [54]:
X_train[0]

array([ 100,  152,   11,  121,    3,   45, 1674,   10,   24,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], d

In [55]:
y_train[20:30]

array([0, 0, 0, 0, 2, 2, 2, 2, 2, 2])

# Save The Training Dataset

In [56]:
from pickle import dump

In [57]:
# Dump X_train into a pickle file
with open("/content/drive/MyDrive/Colab Notebooks/Sentiment Anyalasis/X_train.pkl", "wb") as f:
    dump(X_train, f)

In [58]:
# Dump y_train into a pickle file
with open("/content/drive/MyDrive/Colab Notebooks/Sentiment Anyalasis/y_train.pkl", "wb") as f:
    dump(y_train, f)